In [30]:
from pyscf import gto, scf, fci
import numpy as np
import matplotlib.pyplot as plt
import qiskit
from qiskit import QuantumCircuit
from matplotlib.ticker import ScalarFormatter
from scipy.optimize import minimize
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.circuit.library import TwoLocal
from qiskit_ionq import IonQProvider
from qiskit.quantum_info import Statevector
import os
from qiskit.quantum_info import Pauli
from qiskit_algorithms import NumPyEigensolver
from qiskit.circuit.library import HGate , SdgGate
from qiskit import transpile
from scipy.optimize import differential_evolution
from qiskit_aer import AerSimulator, AerProvider

In [31]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()
  qubit_op = mapper.map(second_q_op)
  return qubit_op , mapper
# 고전값 비교용 FCI 
# H2 인경우로만 해뒀고, 거리 주면 에너지 계산할 수 있음. 
class ConvergenceReached(Exception):
    pass

def hamming_distance(a: str, b: str) -> int:
    return bin(int(a, 2) ^ int(b, 2)).count('1')

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
# cost_func 외부에 전역 상태 저장
convergence_state = {
    "prev_cost": None,
    "streak": 0,
    "tol": 1e-4,        # ε: 변화 허용치
    "patience": 5,      # k: 연속 허용 횟수
    "trigger_stop": False,
}
def early_stop_callback(xk, convergence):
    return convergence_state["trigger_stop"]  # True 반환 시 중단됨



def cost_func(parameter, second_q_op, n_ptl, ansatz, backend,k):
    global convergence_state
    global cost_history_dict 
    params = ansatz.parameters 
    param_dict = dict(zip(params, parameter))
    qc = ansatz.assign_parameters(param_dict)
    qc.measure_all()
    job = backend.run(qc, shots=2000)
    result = job.get_probabilities()
    all_basis = []
    all_prob = []
    for bitstring, prob in result.items():
        count_ones = bitstring.count('1')
        half = len(bitstring) // 2
        left_ones = bitstring[:half].count('1')
        right_ones = bitstring[half:].count('1')
        if count_ones == n_ptl and left_ones==right_ones :
            all_basis.append(bitstring)
            all_prob.append(prob)

    all_basis = np.array(all_basis)
    all_prob = np.array(all_prob)
    top_indices = np.argsort(-all_prob)[:k]
    probable_basis = all_basis[top_indices]
    print("most probable_basis : ", probable_basis)

    n = len(probable_basis)
    H = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            basis_bra = probable_basis[i]
            basis_ket = probable_basis[j]
            if hamming_distance(basis_bra, basis_ket) <= 2:
                print(hamming_distance(basis_bra, basis_ket))
                inner_product = projection_hamiltonian(second_q_op, basis_bra , basis_ket)
                H_ij = np.real(inner_product)
                H[i, j] = H_ij 
            else : 
                H[i, j] = 0
            print("<{}|H|{}> = {}".format(basis_bra,basis_ket,H_ij))
    eigvals, eigvecs = np.linalg.eig(H)
    Energy = np.min(eigvals)

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(Energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {Energy}]")

    prev = convergence_state["prev_cost"]
    if convergence_state["streak"] >= convergence_state["patience"]:
        print("✅ 조기 수렴 조건 충족: 최적화 강제 종료")
        raise ConvergenceReached()

    if prev is not None and abs(Energy - prev) < convergence_state["tol"]:
        convergence_state["streak"] += 1
    else:
        convergence_state["streak"] = 0
    convergence_state["prev_cost"] = Energy

    if convergence_state["streak"] >= convergence_state["patience"]:
        convergence_state["trigger_stop"] = True
        print("✅ 조기 수렴 조건 충족: 최적화 중단 예정")


    return Energy
def parsing(ferm_op):
    one_body_terms = []
    two_body_terms = []

    for label, coeff in ferm_op.items():
        ops = label.split()
        indices = [int(op[2:]) for op in ops]

        if len(ops) == 2:
            one_body_terms.append((coeff, indices))
        elif len(ops) == 4:
            two_body_terms.append((coeff, indices))
        # 그 외 (예: 항이 없는 상수) 등은 무시하거나 추가적으로 처리 가능

    return one_body_terms, two_body_terms

def projection_hamiltonian(second_q_op, basis_bra, basis_ket):
    single_terms, double_terms = parsing(second_q_op)
    dist = hamming_distance(basis_bra, basis_ket)

    Energy = 0


    if dist ==0 : 
        for items in single_terms:
            p = items[1][0]
            q = items[1][1]
            if basis_bra[p] == str(1) and basis_ket[q] == str(1) :
                Energy += np.real(items[0])

        for items in double_terms:
            p = items[1][0]
            q = items[1][1]
            r = items[1][2]
            s = items[1][3]
            if basis_bra[p] == str(1) and basis_bra[q] == str(1) and basis_ket[r] == str(1) and basis_ket[s] == str(1) :
                Energy += np.real(items[0])

    elif dist == 1 :
    elif dist == 2 :


    for items in single_terms:
        p = items[1][0]
        q = items[1][1]
        if basis_bra[p] == str(1) and basis_ket[q] == str(1) :
            Energy += np.real(items[0])

    for items in double_terms:
        p = items[1][0]
        q = items[1][1]
        r = items[1][2]
        s = items[1][3]
        if basis_bra[p] == str(1) and basis_bra[q] == str(1) and basis_ket[r] == str(1) and basis_ket[s] == str(1) :
            Energy += np.real(items[0])
    return Energy
    #for item in pauli_terms:

In [32]:
basis_bra = '10110110110111'
basis_ket = '11010110111110'

hamming_distance(basis_bra, basis_ket) 

4

In [33]:
api_key = "7IMO85iuzmzAAgri4osG5BPTeCFXrKoM"
provider = IonQProvider(api_key)
backend = provider.get_backend("simulator")
#simulator_backend = provider.get_backend("qpu.forte-1")

In [34]:


# To run on hardware, select the backend with the fewest number of jobs in the queue
#service = QiskitRuntimeService(channel="ibm_quantum", token ='55ac54cb4cbfe378d8604606bacfcab38ffd165ecdb215742e64495a259fa53f0ed9e63df5b3df4489d422865a007d9cc981fcb2caffd9e76f17d94d734a8593')
#backend = service.least_busy(operational=True, simulator=False)
#backend = GenericBackendV2(num_qubits=5)

#backend = AerSimulator(method = "statevector",noise_model=None)

In [35]:
O = (0.000, 0.000, 0.000)

# O–H bond length
bond_length = 0.958  # in angstroms
angle_deg = 104.5
angle_rad = np.deg2rad(angle_deg / 2)

# 좌우 대칭 구조로 H 원자 배치
H1 = (bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)
H2 = (-bond_length * np.sin(angle_rad), bond_length * np.cos(angle_rad), 0.0)

# 최종 입력
atoms = ["O", "H", "H"]
coords = [O, H1, H2]
basis = "sto3g"
charge = 0
multiplicity = 1

Co_O_moleculeinfo = MoleculeInfo(atoms, coords, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver.from_molecule(Co_O_moleculeinfo, basis=basis)
E_problem = driver.run() # 여기는 이후, As_transformer 로 변경. 

fermionic_hamiltonian = E_problem.hamiltonian
second_q_op = fermionic_hamiltonian.second_q_op()
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
hamiltonian, mapper = fermion_to_qubit(E_problem, second_q_op, "JW")
H_op = hamiltonian.to_operator()
H_matrix = H_op.data
num_qubits = hamiltonian.num_qubits
num_particles = E_problem.num_particles
num_electrons = np.sum(num_particles)
num_spatial_orbitals = E_problem.num_spatial_orbitals


init_state = HartreeFock(num_spatial_orbitals,num_particles, mapper)
ansatz = TwoLocal(num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=init_state).decompose()
#ansatz = UCCSD(num_spatial_orbitals,num_particles,mapper,initial_state=init_state)

pauli_basis = hamiltonian.paulis
coeffs = hamiltonian.coeffs
num_params = ansatz.num_parameters

""""
x0 = 2 * np.pi * np.random.random(num_params)

res = minimize(
    cost_func,
    x0,
    args=(second_q_op, num_electrons, ansatz, simulator_backend, 4),
    method="cobyla",
)

"""

bounds = [(0, 2*np.pi)] * num_params  # ansatz 파라미터 수에 따라 조절

try:
    res = differential_evolution(
        func=lambda w: cost_func(w, second_q_op, num_electrons, ansatz, backend,20),
        bounds=bounds,
        callback=early_stop_callback,
        polish=True,
    )
except ConvergenceReached:
    print("🔁 최적화가 조기 수렴 조건에 따라 종료되었습니다.")

plt.plot(range(cost_history_dict["iters"]), cost_history_dict["cost_history"] + repulsion, label='E_VQE')
plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=False))
plt.xlabel("Iterations (#)")
plt.ylabel("Energy (Hartree)")
plt.title("Energy Calc. for each iteration")
plt.legend()
plt.grid()
plt.draw()

most probable_basis :  ['11101011011011' '10111011011011' '01110110111101' '11011011011011'
 '00111111010111' '11110011111010' '01111101110011' '11110011101110'
 '11110011111001' '11011101010111' '01011111011101' '11011011010111'
 '11101011101011' '10110110111011' '11101011010111' '11100111111100'
 '11110011110101' '11101011101110' '11110011001111' '11101101101101']
0
<11101011011011|H|11101011011011> = -69.23665040156949
2
<11101011011011|H|10111011011011> = -69.86793154601507
<11101011011011|H|01110110111101> = -69.86793154601507
2
<11101011011011|H|11011011011011> = -70.63316280324231
<11101011011011|H|00111111010111> = -70.63316280324231
<11101011011011|H|11110011111010> = -70.63316280324231
<11101011011011|H|01111101110011> = -70.63316280324231
<11101011011011|H|11110011101110> = -70.63316280324231
<11101011011011|H|11110011111001> = -70.63316280324231
<11101011011011|H|11011101010111> = -70.63316280324231
<11101011011011|H|01011111011101> = -70.63316280324231
<11101011011011|H|11

/Users/yoonho/Documents/Cotton/lib/python3.9/site-packages/qiskit_ionq/helpers.py:664: UserWarning: Retrying retrieve_job after IonQRetriableError('Unknown error')
  warnings.warn(


most probable_basis :  ['10111101111100' '10101111100111' '10110111011101' '00111111010111'
 '10111101011101' '10111011111100' '10011111111100' '10111011110101'
 '10110111011110' '10110111001111' '10011111011101' '10101111001111'
 '00111111001111' '10011111100111' '00111111100111' '10111101010111'
 '10110111110101' '10111101011110' '10101111010111' '10011111110101']
0
<10111101111100|H|10111101111100> = -71.59333240415307
<10111101111100|H|10101111100111> = -71.59333240415307
<10111101111100|H|10110111011101> = -71.59333240415307
<10111101111100|H|00111111010111> = -71.59333240415307
2
<10111101111100|H|10111101011101> = -71.41567295293596
2
<10111101111100|H|10111011111100> = -72.1651651605634
2
<10111101111100|H|10011111111100> = -71.41321289754893
<10111101111100|H|10111011110101> = -71.41321289754893
<10111101111100|H|10110111011110> = -71.41321289754893
<10111101111100|H|10110111001111> = -71.41321289754893
<10111101111100|H|10011111011101> = -71.41321289754893
<10111101111100|H|1

ReadTimeout: HTTPSConnectionPool(host='api.ionq.co', port=443): Read timed out. (read timeout=30)